In [34]:
import torch
import torch.nn as nn
import numpy as np 
import torch.nn.functional as F 

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [35]:
class fxnnxc(nn.Module):
    def __init__(self):
        super(fxnnxc, self).__init__()
        self.linear1 = nn.Linear(28*28, 256)
        self.linear2 = nn.Linear(256, 64)
        self.linear3 = nn.Linear(64, 10)

    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = self.linear3(x)
        return x

In [36]:
#--------------------------------------------- file read
x_train = np.load("data/mnist_train.npy")
x_test  = np.load("data/mnist_test.npy")
y_train = np.load("data/mnist_train_target.npy")
y_test  = np.load("data/mnist_test_target.npy")
#--------------------------------------------- numpy to tensor
x_train  = torch.from_numpy(x_train).float()       #long으로 하면 loss 계산할 때 에러
x_test   = torch.from_numpy(x_test).float()
y_train  = torch.from_numpy(y_train).long()        #float으로 하면 loss 계산할 때 에러  
y_test   = torch.from_numpy(y_test).long()

#--------------------------------------------- data to dataset
train_dataset = torch.utils.data.TensorDataset(x_train, y_train)
test_dataset  = torch.utils.data.TensorDataset(x_test,  y_test)

#--------------------------------------------- dataset to dataloader 
train_loader = torch.utils.data.DataLoader(train_dataset,
                                          batch_size=128,
                                          shuffle=True,
                                          num_workers=2)
                                    
test_loader = torch.utils.data.DataLoader(train_dataset,
                                          batch_size=128,
                                          shuffle=True,
                                          num_workers=2)                                 
                                    

In [37]:
model = fxnnxc()

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

In [38]:
EPOCH = 1

for t in range(EPOCH): # EPOCH
    for i, (sample, target) in enumerate(train_loader): #BATCH
        sample = sample.view(sample.size()[0], -1)   # X 데이터가 28,28 사이즈라서 768 사이즈로 변환해주는 부분   
        y = model(sample)
        loss = criterion(y, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if i % 100 == 99:
            print(t, loss.item())      
        if i==0:      
            writer.add_graph(model, sample)
        writer.add_scalar("Loss/train", loss.item(), i)

0 1.8397842645645142


In [39]:
# Test

correct = 0
total = 0
with torch.no_grad(): 
    for data in test_loader:
        images, labels = data
        images = images.view(images.size()[0], -1)   
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

In [40]:
print(f'Accuracy of the network on the 10000 test images: {100 * correct / total}')

Accuracy of the network on the 10000 test images: 79.57833333333333
